### Hypothesis

$ H(X) = \frac{1}{1 + exp(-W^TX)} $

### Cost

$ cost(W) = \frac{-1}{m}\sum ylog(H(x) + (1 - y)(log(1 - H(x)) $

## Import

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
#For reproducibility
torch.manual_seed(1)

## Training Data

In [91]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]  
y_data = [[0], [0], [0], [1], [1], [1]]  

In [92]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)
print(x_train)
print(y_train)
print(x_train.shape)
print(y_train.shape)

tensor([[1., 2.],
        [2., 3.],
        [3., 1.],
        [4., 3.],
        [5., 3.],
        [6., 2.]])
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.]])
torch.Size([6, 2])
torch.Size([6, 1])


## Computing the Hypothesis

In [6]:
print('e^1 equals: ', torch.exp(torch.FloatTensor([1])))

e^1 equals:  tensor([2.7183])


In [8]:
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [13]:
#hypothesis = 1 / (1 + torch.exp(-(x_train.matmul(W) + b)))  #torch.matmul(x,w)
hypothesis = torch.sigmoid(x_train.matmul(W) + b)

In [14]:
print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward>)
torch.Size([6, 1])


## Computing the Cost Function

In [15]:
print(hypothesis)
print(y_train)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward>)
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.]])


In [16]:
losses = -(y_train * torch.log(hypothesis) + (1 - y_train[0]) * torch.log(1 - hypothesis))
print(losses)

tensor([[0.6931],
        [0.6931],
        [0.6931],
        [1.3863],
        [1.3863],
        [1.3863]], grad_fn=<NegBackward>)


In [17]:
cost = losses.mean()
print(cost)

tensor(1.0397, grad_fn=<MeanBackward0>)


In [18]:
F.binary_cross_entropy(hypothesis, y_train)

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward>)

## Whole Training Procedure

In [19]:
# 모델 초기화
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=1)

nb_epochs = 1999
for epoch in range(nb_epochs + 1):
    
    # Cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(W) + b) 
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad() #초기화 안하면 기존의 gradient에 더하게 됨
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:3d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch   0/1999 Cost: 0.693147
Epoch 100/1999 Cost: 0.134722
Epoch 200/1999 Cost: 0.080643
Epoch 300/1999 Cost: 0.057900
Epoch 400/1999 Cost: 0.045300
Epoch 500/1999 Cost: 0.037261
Epoch 600/1999 Cost: 0.031672
Epoch 700/1999 Cost: 0.027556
Epoch 800/1999 Cost: 0.024394
Epoch 900/1999 Cost: 0.021888
Epoch 1000/1999 Cost: 0.019852
Epoch 1100/1999 Cost: 0.018165
Epoch 1200/1999 Cost: 0.016743
Epoch 1300/1999 Cost: 0.015528
Epoch 1400/1999 Cost: 0.014479
Epoch 1500/1999 Cost: 0.013562
Epoch 1600/1999 Cost: 0.012755
Epoch 1700/1999 Cost: 0.012039
Epoch 1800/1999 Cost: 0.011400
Epoch 1900/1999 Cost: 0.010825


# Evaluation

In [20]:
prediction = hypothesis >= torch.FloatTensor([0.5])
print(prediction[:5])

tensor([[0],
        [0],
        [0],
        [1],
        [1]], dtype=torch.uint8)


In [21]:
print(prediction[:5])
print(y_train[:5])

tensor([[0],
        [0],
        [0],
        [1],
        [1]], dtype=torch.uint8)
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.]])


In [22]:
correct_prediction = prediction.float() == y_train
print(correct_prediction[:5])

tensor([[1],
        [1],
        [1],
        [1],
        [1]], dtype=torch.uint8)


## Higher Implementation with Class

In [93]:
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2, 1)  # self.linear = {w, b}  W는 6*1사이즈, b는 1사이즈
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        return self.sigmoid(self.linear(x))

In [94]:
model = BinaryClassifier()

In [95]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr= 1)  #parameter는 W,b

nb_epochs = 100
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    hypothesis = model(x_train)
    
    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    if epoch % 10 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_train
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}$'.format(epoch, nb_epochs, cost.item(), accuracy * 100))

Epoch    0/100 Cost: 0.858420 Accuracy 50.00$
Epoch   10/100 Cost: 0.555261 Accuracy 83.33$
Epoch   20/100 Cost: 0.573137 Accuracy 83.33$
Epoch   30/100 Cost: 0.463877 Accuracy 83.33$
Epoch   40/100 Cost: 0.378501 Accuracy 83.33$
Epoch   50/100 Cost: 0.298489 Accuracy 83.33$
Epoch   60/100 Cost: 0.228177 Accuracy 83.33$
Epoch   70/100 Cost: 0.178263 Accuracy 100.00$
Epoch   80/100 Cost: 0.153821 Accuracy 100.00$
Epoch   90/100 Cost: 0.141763 Accuracy 100.00$
Epoch  100/100 Cost: 0.132282 Accuracy 100.00$
